In [3]:
import os
import re
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import numpy as np
import pandas as pd
import json
import pymorphy2
import nltk
from tqdm import tqdm
from vk_api_dir.vk_api.vk_api_me import API_from_my_data

## Автоматизация репостов - первые шаги

Цель: научиться по аудитории группы искать сообщества, подписываться на них и автоматически репостить самые успешные посты

(created by data2you)

## Часть 1
### Интересы пользователей группы

- для каждого пользователя скачиваем массив интересов
- для каждого интереса(слова) приводим его к нормальной форме (например pymorphy)
- подсчет популярных.

In [4]:
group = 'okean_music'
api = API_from_my_data.get_api()

password: 


In [393]:
def _parse_and_normalize(s):
    s = nltk.word_tokenize(s)
    morph = pymorphy2.MorphAnalyzer()
    s = [morph.parse(i)[0].normal_form for i in s]
    return s
def check(x):
    return not (x in [u'на', u'под', u'и', u'c', u'в', u'not', u'с'])
def get_n_interests(api, n, group):
    """
    получаем n самых популярныз интересов в данной группе
    """
    # my api registration
    _dict = api.groups.getMembers(group_id=group)
    count, members = _dict['count'], _dict['users']
    if len(members) >= 1000:
        raise "Not Implemented"
    
    _dict = api.users.get(user_ids=members, fields='interests')
    all_strs = []
    for _ in _dict:
        if 'interests' in _ and len(_['interests']) > 0:
            all_strs.append(_['interests'])
    interest_to_count = dict()
    for _interest in tqdm(np.concatenate([_parse_and_normalize(i) for i in all_strs])):
        if not _interest in interest_to_count:
            interest_to_count[_interest] = 0
        interest_to_count[_interest] += 1
    # костыльно оставляем только слова и сортируем по частоте
    out = sorted(filter(lambda x: x[0].isalpha() and check(x[0]), interest_to_count.items()), key=lambda x: x[1], reverse=True)
    for i in out:    
        print i[0], i[1]
    return map(lambda x: x[0], out[:n])
interests = get_n_interests(api, 10, group)

100%|██████████| 718/718 [00:00<00:00, 402931.53it/s]

музыка 17
кино 8
любить 5
фотография 5
театр 5
искусство 5
конкурс 4
спорт 4
литература 4
книга 3
философия 3
рукоделие 3
психология 3
танец 3
волейбол 2
аниме 2
наука 2
концерт 2
интересный 2
автомобиль 2
хороший 2
music 2
рисование 2
видеоарт 2
баскетбол 2
живопись 2
слушать 2
кошка 2
танцевать 2
буккроссинг 1
дождь 1
фольклор 1
журналинг 1
саморазвитие 1
список 1
классика 1
отсутствовать 1
медицина 1
прагматизм 1
посещать 1
действие 1
идея 1
этика 1
production 1
общественный 1
asumiko 1
алхимия 1
coffee 1
рок 1
график 1
ведущий 1
япония 1
чтение 1
минимализм 1
самопознание 1
скрытый 1
затягивать 1
palahniuk 1
природа 1
chuck 1
asano 1
режим 1
друг 1
nakamura 1
poppy 1
хотеть 1
машинка 1
明日美子 1
любовь 1
нота 1
brite 1
art 1
семья 1
японский 1
books 1
дартс 1
английский 1
интернет 1
ходить 1
печатный 1
велосипед 1
играть 1
звук 1
объять 1
огромный 1
скука 1
дизайн 1
камера 1
хобби 1
свой 1
необъятный 1
участвовать 1
воплощение 1
outfit 1
самый 1
составлять 1
фильм 1
спать 1
кинофильм 

## Часть 2
### Поиск сообществ

теперь когда мы знаем топ n интересов подписчиков группы, можно взять эти интересы и найти по ним много-много людей, у которых есть свои подписки.
Итак на итерации мы

- берем случайный поднабор интересов и ищем человека
- получаем группы на которые он подписан;

Затем находим топ k групп, наиболее популярных среди этой подписоты


In [6]:
import random
import time
import vk

In [7]:
VkAPIError = vk.exceptions.VkAPIError

In [8]:
def get_users_by_interests(api, interests, num):
    # стандартная вкшная переменная
    offs = 0
    persons_found = 0
    NUM_INTERESTS = 5
    users = []
    while persons_found < num:
        #  https://vk.com/dev/users.search
        # берем сколько-то случайных интересов из списка
        curr_interests = [random.choice(interests) for i in range(NUM_INTERESTS)]
        _dict = api.users.search(offset=offs, interests=','.join(curr_interests))
        if _dict[0] + persons_found < num:
            raise Exception("To taught search parameters: only {0} users found".format(_dict[0] + persons_found))
        _dict = _dict[1:]
        persons_found += len(_dict)
        for i in [_usr['uid'] for _usr in _dict]:
            users.append(i)
        time.sleep(1)
    return users
typical_user_ids = get_users_by_interests(api, interests, num=100)
typical_user_ids

[13821502,
 216970122,
 346271964,
 209490370,
 258468989,
 29470165,
 38942884,
 167041343,
 92364235,
 263410754,
 85599545,
 181949463,
 20420307,
 280153768,
 203317835,
 8027438,
 855775,
 45854551,
 189539499,
 56478071,
 13821502,
 216970122,
 22077894,
 307134457,
 346271964,
 59605782,
 209490370,
 258468989,
 29470165,
 137735663,
 68303934,
 38942884,
 228929867,
 181677785,
 160973233,
 92364235,
 244620136,
 263410754,
 85599545,
 360291582,
 12489927,
 151576949,
 11344503,
 107523643,
 231817577,
 68853510,
 174577275,
 92528955,
 193028632,
 255044186,
 48393413,
 314042609,
 33814534,
 43053589,
 102975176,
 205886807,
 177028393,
 402863357,
 12489927,
 160489574,
 20176333,
 11344503,
 107523643,
 194172492,
 195917130,
 68853510,
 174577275,
 303256288,
 92528955,
 13821502,
 11230885,
 193028632,
 255044186,
 314042609,
 33814534,
 43053589,
 210841413,
 209452406,
 231817577,
 13821502,
 216970122,
 22077894,
 307134457,
 155446937,
 252356094,
 346271964,
 596057

Отлично. Мы получили id пользовательй со схожими интересами. Достанем из подписки и найдем наиболее популярные.

In [398]:
def get_n_groups_by_ids(api, ids, n=10):
    popular = dict()
    for user_id in tqdm(ids):
        OK = False
        while not OK:
            try:
                _groups = api.groups.get(user_id=user_id)
                OK = True
            except Exception, e:
                print(e)
                time.sleep(0.3)
        for _group_id in _groups:
            _group_id = int(_group_id)
            if not _group_id in popular:
                popular[_group_id] = 0
            popular[_group_id] += 1
        time.sleep(0.3)
    out = sorted(popular.items(), key=lambda x: x[1], reverse=True)
    return out
popular_groups_ = get_n_groups_by_ids(api, typical_user_ids)

 28%|██▊       | 32/116 [00:43<02:15,  1.62s/it]

HTTPSConnectionPool(host='api.vk.com', port=443): Read timed out. (read timeout=10)


 57%|█████▋    | 66/116 [01:30<01:13,  1.46s/it]

HTTPSConnectionPool(host='api.vk.com', port=443): Read timed out. (read timeout=10)
HTTPSConnectionPool(host='api.vk.com', port=443): Max retries exceeded with url: /method/groups.get (Caused by ConnectTimeoutError(<requests.packages.urllib3.connection.VerifiedHTTPSConnection object at 0x11aaed850>, 'Connection to api.vk.com timed out. (connect timeout=10)'))


100%|██████████| 116/116 [03:35<00:00,  1.34s/it]


In [400]:
popular_groups = popular_groups_[:100]
# Важно! во многих функциях id группы vk отрицательный!
# не будем жадничать

сразу уберем MDK :) и вообще эта функция будет только расширяться

In [405]:
def preprocess(popular_groups):
    good_ = []
    blocked = {-12382740, -57846937}
    for group in tqdm(popular_groups):
        if group in blocked:
            continue
        group = -abs(group[0])
        #  проверим, что в группе есть посты
        OK = False
        while not OK:
            try:
                req_ = api.wall.get(owner_id=group)
                if req_[0] > 10:
                    good_.append(group)
                OK = True
            except Exception, e:
                print(e)
                time.sleep(0.3)
    return good_

In [406]:
checked_groups = preprocess(popular_groups)

 46%|████▌     | 46/100 [00:38<01:05,  1.21s/it]

('Connection aborted.', error("(54, 'ECONNRESET')",))


 51%|█████     | 51/100 [00:45<01:07,  1.38s/it]

('Connection aborted.', error("(54, 'ECONNRESET')",))


100%|██████████| 100/100 [01:32<00:00,  1.42it/s]


In [407]:
len(checked_groups)

100

In [409]:
api.groups.getById(group_ids=checked_groups)

VkAPIError: 100. One of the parameters specified was missing or invalid: group_ids is undefined. request_params = {u'oauth': u'1', u'group_ids': u'-22822305,-28556858,-22079806,-57846937,-12382740,-19542789,-1415705,-31480508,-43215063,-56106344,-24713873,-48319873,-27895931,-25276999,-29246653,-49840023,-28477986,-29302425,-11982368,-22798006,-34483558,-34384434,-29573241,-31976785,-23471538,-26750264,-35034571,-36775802,-33393308,-31072900,-65960786,-48940689,-23433159,-34229261,-30559917,-50106785,-34137285,-2158488,-40567146,-29559271,-25397178,-47679753,-632045,-45595714,-28950133,-460389,-34491673,-12353330,-30179569,-23148107,-29178527,-40587282,-34168005,-39153701,-23170931,-45588871,-44554509,-23616160,-65539551,-74192244,-30022666,-24098496,-101209760,-35807216,-15755094,-49849615,-34529402,-25232578,-63133420,-37862023,-43776215,-28446706,-34298047,-26614831,-62398004,-29544671,-35061290,-41883468,-31545235,-23164653,-57680408,-29258893,-34191936,-31577711,-23337480,-3972880

Посчитаем сколько вообще в данной группе набирается лайков в среднем на одну сек времени
Заодно создадим класс групп vk

In [410]:
import time
int(time.time())

1494959998

In [419]:
class Post:
    def __init__(self, owner_id, post_id, rating):
        self.owner_id = owner_id
        self.post_id = post_id
        self.rating = rating

class Group:
    def __init__(self, api, gid):
        # id группы
        self.gid = gid
        self.api = api
        self.posts = []
        # сколько вообще в данной группе набирается лайков в среднем на одну секунду времени
        self.rating = 0
    
    def get_n_posts(self, n):
        #  обновление данных о постах
        self.time = time.time()
        #  offset = 10 чтобы не брать посты совсем уж без лайков
        #  достаем посты для данной группы
        OK = False
        while not OK:
            try:
                self.posts = api.wall.get(owner_id=self.gid, count=n, extended=1, filter='owner', version='5.64')['wall']
                OK = True
            except Exception, e:
                print "Error", e
        self.num, self.posts = self.posts[0], self.posts[1:]
        return len(self.posts) == n
    
    def calc_rate(self, new_posts=True):  #  сколько вообще в данной группе набирается лайков в среднем на одну минуту времени   
        if new_posts:
            self.get_n_posts(10)
        # посчитаем лайки
        mid_ans = 0   
        for i in self.posts:
            LIKES = int(i['likes']['count'])
            mid_ans += LIKES / (self.time - i['date'])
            #  it's needed
            time.sleep(0.3)
        mid_ans /= len(self.posts)  
        self.rating = mid_ans
        return self.rating != 0
    def calc_post_rating(self, post, rating):
        likes_cnt = int(post['likes']['count'])
        rate = likes_cnt / (self.time - post['date'])
        return rate / rating
    def get_rated_posts(self):
        # get posts from group and evaluate them
        posts = []
        for post in self.posts:
            post_rating = self.calc_post_rating(post, self.rating)
            posts.append(Post(self.gid, post['id'], post_rating))
        return posts
        

In [420]:
api.wall.get(owner_id="-57846937", count=2, extended=1, filter='owner', version='5.64')['wall'][1]['id']

13627372

In [421]:
buisness = Group(api, "-28556858")
buisness.get_n_posts(10)
# вычисляем рейтинг группы по хранящимся у нас постам
buisness.calc_rate()
rated_posts = buisness.get_rated_posts()

Error 502 Server Error: Bad Gateway for url: https://api.vk.com/method/wall.get


In [414]:
print buisness.rating

0.0630843712746


In [415]:
#  например можно посмотреть рейтинг конкретного поста
print "Post id = {}, rating = {}".format(rated_posts[1].post_id, rated_posts[1].rating)
print "Post id = {}, rating = {}".format(rated_posts[3].post_id, rated_posts[3].rating)

Post id = 167483, rating = 0.163491239811
Post id = 167481, rating = 1.83549398485


теперь на напишем класс Robot, который будучи запущенным будет реализовывать интерфейс подписки на выбранные нами паблики

Важно не словить 'VkAPIError'!

Необходимый функционал:
- [получить](https://vk.com/dev/wall.get) свежие посты из группы
- [получить](https://vk.com/dev/likes.getList) число лайков -> рейтинг
- [репостнуть](https://vk.com/dev/wall.repost)

In [416]:
checked_groups = map(lambda x: -abs(x), checked_groups)
checked_groups

[-22822305,
 -28556858,
 -22079806,
 -57846937,
 -12382740,
 -19542789,
 -1415705,
 -31480508,
 -43215063,
 -56106344,
 -24713873,
 -48319873,
 -27895931,
 -25276999,
 -29246653,
 -49840023,
 -28477986,
 -29302425,
 -11982368,
 -22798006,
 -34483558,
 -34384434,
 -29573241,
 -31976785,
 -23471538,
 -26750264,
 -35034571,
 -36775802,
 -33393308,
 -31072900,
 -65960786,
 -48940689,
 -23433159,
 -34229261,
 -30559917,
 -50106785,
 -34137285,
 -2158488,
 -40567146,
 -29559271,
 -25397178,
 -47679753,
 -632045,
 -45595714,
 -28950133,
 -460389,
 -34491673,
 -12353330,
 -30179569,
 -23148107,
 -29178527,
 -40587282,
 -34168005,
 -39153701,
 -23170931,
 -45588871,
 -44554509,
 -23616160,
 -65539551,
 -74192244,
 -30022666,
 -24098496,
 -101209760,
 -35807216,
 -15755094,
 -49849615,
 -34529402,
 -25232578,
 -63133420,
 -37862023,
 -43776215,
 -28446706,
 -34298047,
 -26614831,
 -62398004,
 -29544671,
 -35061290,
 -41883468,
 -31545235,
 -23164653,
 -57680408,
 -29258893,
 -34191936,
 -3157771

In [424]:
class Robot:
    def __init__(self, api, groups):
        """
        api: vkapi session
        groups: Group class'
        
        Class Robot takes Posts from Groups 
        and chose the best Post via comparator 
        and repost this Post in chosen user's group via Poster
        """
        self.api = api
        if len(groups) > 100:
            raise Exception("Too much...")
        self.groups = []
        for g_id in groups:
            self.groups.append(Group(api, g_id))
        #  ban bad groups    
        self.banned = set()
    def get_all_rated_posts(self):
        self.rated_posts = []
        for group in self.groups:
            print group.gid
            if not (group.gid in self.banned) and group.get_n_posts(10) and group.calc_rate():
                print group.rating
                if group.rating > 0.0005:
                    self.rated_posts.append(group.get_rated_posts())
                else:
                    self.banned.add(group.gid)
                    print ":/"
            else : 
                print ":("
                self.banned.add(group.gid)
    def get_max_rated_post(self):
        self.get_all_rated_posts()
        mx_ans = Post(-1 ,-1, -1)
        for batch in self.rated_posts:
            mx = max(batch, key=lambda x: x.rating)
            mx_ans = max(mx, mx_ans, key=lambda x: x.rating)
        return mx_ans
    

r = Robot(api, checked_groups[:2])   
winner = r.get_max_rated_post()

-22822305
0.0022512998302
-28556858
0.0552948460981
[[<__main__.Post instance at 0x116a42b90>, <__main__.Post instance at 0x11334c710>, <__main__.Post instance at 0x113347518>, <__main__.Post instance at 0x113347488>, <__main__.Post instance at 0x113347758>, <__main__.Post instance at 0x11272af38>, <__main__.Post instance at 0x11272d170>, <__main__.Post instance at 0x113311200>, <__main__.Post instance at 0x113311ef0>, <__main__.Post instance at 0x113311bd8>], [<__main__.Post instance at 0x113311098>, <__main__.Post instance at 0x113329758>, <__main__.Post instance at 0x113347830>, <__main__.Post instance at 0x1151405a8>, <__main__.Post instance at 0x115140bd8>, <__main__.Post instance at 0x1151405f0>, <__main__.Post instance at 0x115140440>, <__main__.Post instance at 0x1151403f8>, <__main__.Post instance at 0x115140290>, <__main__.Post instance at 0x115140560>]]


In [425]:
print winner.owner_id, winner.post_id, winner.rating

-28556858 167482 2.3482917569


In [427]:
class Poster:
    def __init__(self, post):
        access_token="f850cf72b2adcc5d54697a8a74cd82ef87ab671419d4cb453bd939f7fdae933252c2ac448e037672442b3"
        session = vk.AuthSession(access_token=access_token)
        api = vk.API(session)
        # self.post = post
        self.api = api
        self.post = post
    def repost(self):
        #  self.api.wall.post(owner_id=-22183949, message="LOL!")
        self.api.wall.repost(object="wall{}_{}".format(self.post.owner_id, self.post.post_id), group_id=22183949)

In [428]:
poster1 = Poster(winner)

In [429]:
poster1.repost()